In [ ]:
!pip install selenium
!apt-get update # to update ubuntu to correctly run apt install
!apt install chromium-chromedriver
!cp /usr/lib/chromium-browser/chromedriver /usr/bin

In [ ]:
import numpy as np 
import matplotlib.pyplot as plt 
import pandas as pd
import os
import re 
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import time
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.by import By
import urllib3
from bs4 import BeautifulSoup
import csv
from selenium.webdriver.common.action_chains import ActionChains
import requests 
from urllib import request
import sys
from selenium import webdriver
from google.colab import drive
from google.colab import files

In [ ]:
from google.colab import drive 
drive.mount('/content/gdrive')
df=pd.read_csv('gdrive/My Drive/Colab Notebooks/FYP/MarketWatch List.csv')

In [ ]:
company_names=[]
for index, row in df.iloc[290:300].iterrows():
  sys.path.insert(0,'/usr/lib/chromium-browser/chromedriver')
  chrome_options = webdriver.ChromeOptions()
  chrome_options.add_argument('--headless')
  chrome_options.add_argument('--no-sandbox')
  chrome_options.add_argument('--disable-dev-shm-usage')
  driver = webdriver.Chrome('chromedriver',options=chrome_options)
  count=0
  company_name=row['description']
  url = row['url']
  count = count + 1

  if not pd.isna(url):
  
    company_names.append(company_name)

    print(company_name)
    print(url)
    print(count)
    print()

    driver.get(url)
    wait = WebDriverWait(driver, 20)
    time.sleep(5)
    element = driver.find_element_by_xpath('/html/body/div[3]/div[6]/div[2]/div[1]/mw-tabs/div[2]/div[1]/mw-scrollable-news-v2')
    for i in range(0, 120):
      driver.execute_script("arguments[0].scrollTop = arguments[0].scrollHeight", element)
      time.sleep(1)
    
    html = driver.page_source
    driver.close()
    soup = BeautifulSoup(html, 'html.parser')
    data1 = soup.find('div', attrs = {'class': 'tab__pane is-active j-tabPane'})
    links = []

    for link in data1.find_all('a'): 
      links.append(link.get('href'))
    
    links = list(dict.fromkeys(links))
    links = [link for link in links if link is not None]
    newlinks = [] 
    for link in links: 
      if link.startswith('https://www.marketwatch.com/story') or link.startswith('https://www.marketwatch.com/article'):
        newlinks.append(link) 
      else:
        continue
    
    locals()["df"+str(company_name)] = pd.DataFrame(columns = ['Date', 'Headline', 'Text'])
    for link in newlinks:
      try: 
        url = link 
        html = request.urlopen(url).read().decode('utf-8')
        soup = BeautifulSoup(html, 'html.parser')
        
        head = soup.find('div', attrs = {'class': 'article__masthead'})
        headline = head.find('h1').string 
        
        if headline == None: 
          head = soup.find('div', attrs = {'class': 'article__masthead'})
          headline = head.find('h1', attrs = {'class': 'article__headline'})
          headline = headline.contents[2]
        
        date = head.find('time', attrs = {'class': 'timestamp timestamp--pub'}).string
        text = soup.find('div', attrs = {'id': 'js-article__body'})
        
        if text == None: 
          continue 
        
        paragraphs = text.find_all('p')
        
        text_final = ""
        for paragraph in paragraphs: 
          text_final = text_final + paragraph.getText()


        # company_name=str(company_name).strip()
        date = " ".join(date.split())
        headline = " ".join(headline.split())
        text_final = " ".join(text_final.split())
        locals()["df2"+str(company_name)] = pd.DataFrame({"Date":[date], "Headline":[headline], "Text":[text_final]})
        locals()["df"+str(company_name)] = locals()["df"+str(company_name)].append(locals()["df2"+str(company_name)])
      
      except:
        pass
    

    locals()["df"+str(company_name)].to_csv('{0}.csv'.format(company_name))

    files.download('{0}.csv'.format(company_name))
    #("df"+str(company_name)).to_csv('{0}.csv'.format(company_name))
    # !cp ("{0}.csv".format(company_name)) "gdrive/My Drive/Colab Notebooks/FYP/MWscrapped"
    # name=("{}.csv".format(company_name))
    # !cp str(name) "gdrive/My Drive/Colab Notebooks/FYP/MWscrapped"